In [1]:
import sys
sys.path
sys.path.append('../../../source_synphot/')
import source_synphot.passband
import source_synphot.io
import source_synphot.source
import astropy.table as at
from collections import OrderedDict
import pysynphot as S
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
%matplotlib notebook

#from zp_fit import *
from scipy.optimize import minimize
from astropy.stats import sigma_clip
import pandas as pd
import warnings
#warnings.filterwarnings("ignore")
def myround(x, prec=2, base=.5):
    return round(base * round(float(x)/base),prec)

In [2]:
passbands = at.Table.read('../../../source_synphot/source_synphot/passbands/pbzptmag.txt',format='ascii')
pbnames = [x['obsmode'] for x in passbands if x['passband'].startswith("TESS")]
pbnames += [x['obsmode'] for x in passbands if x['passband'].startswith("PS1")]
model_mags = 0.
model = 'AB'
pbs = source_synphot.passband.load_pbs(pbnames, model_mags, model)
pbnames = pbs.keys()

21.8904558292275
0.0
21.88238391456467
0.0
20.836195000114955
0.0
21.370261319386927
0.0
21.793287528964058
0.0
22.09903034378283
0.0
22.325516493196154
0.0


In [2]:
models = at.Table.read('../../../source_synphot/ckmodels.txt',format='ascii')
logZ = 0.
model_sed_names = []
temp = []
for s in models:
    teff = max(3500.,s['teff'])
    logg = myround(s['logg'])
    # the models with logg < 1 are just padded with 0s
    if logg >= 1:
        temp.append(teff)
        modstring = 'ckmod{:.0f}_{:.1f}_{:.2f}'.format(teff,logZ, logg)
        model_sed_names.append(modstring)
model_sed = source_synphot.source.load_source(model_sed_names)

../../../source_synphot/source_synphot/source.py:50: RuntimeWarning: Source ckmod44852_0.0_4.00 not loaded
  warnings.warn(message, RuntimeWarning)
../../../source_synphot/source_synphot/source.py:50: RuntimeWarning: Source ckmod42857_0.0_4.00 not loaded
  warnings.warn(message, RuntimeWarning)
../../../source_synphot/source_synphot/source.py:50: RuntimeWarning: Source ckmod40862_0.0_4.00 not loaded
  warnings.warn(message, RuntimeWarning)
../../../source_synphot/source_synphot/source.py:50: RuntimeWarning: Source ckmod39865_0.0_4.00 not loaded
  warnings.warn(message, RuntimeWarning)
../../../source_synphot/source_synphot/source.py:50: RuntimeWarning: Source ckmod40300_0.0_3.50 not loaded
  warnings.warn(message, RuntimeWarning)
../../../source_synphot/source_synphot/source.py:50: RuntimeWarning: Source ckmod39000_0.0_3.00 not loaded
  warnings.warn(message, RuntimeWarning)
../../../source_synphot/source_synphot/source.py:50: RuntimeWarning: Source ckmod34200_0.0_3.00 not loaded
  war

21.8904558292275
21.88238391456467
20.836195000114955
21.370261319386927
21.793287528964058
22.09903034378283
22.325516493196154


In [45]:
color1 = 'ps1g_ps1r'
color2 = 'tessps1_tess'
color3 = 'ps1g_tess'
color4 = 'tess_ps1i'
color5 = 'tess_ps1z'
col1 = []
col2 = []
col3 = []
col4 = []
col5 = []
col6 = []
col7 = []
# construct color-color vectors
g = 'ps1g'
r = 'ps1r'
i = 'ps1i'
z = 'ps1z'
y = 'ps1y'
t = 'tess'
tp = 'tessps1'
for modelname in model_sed:
    model= model_sed[modelname]
    model = S.ArraySpectrum(model.wave, model.flux, name=modelname)
    g, r = color1.split('_')
    pbg, zpg = pbs[g]
    pbr, zpr = pbs[r]
    pbi, zpi = pbs[i]
    pbz, zpz = pbs[z]
    pby, zpy = pbs[y]
    pbt, zpt = pbs[t]
    pbtp, zptp = pbs[tp]
    mg  = source_synphot.passband.synphot(model, pbg, zpg)
    mr  = source_synphot.passband.synphot(model, pbr, zpr)
    mi  = source_synphot.passband.synphot(model, pbi, zpi)
    mz  = source_synphot.passband.synphot(model, pbz, zpz)
    my  = source_synphot.passband.synphot(model, pby, zpy)
    mt  = source_synphot.passband.synphot(model, pbt, zpt)
    mtp = source_synphot.passband.synphot(model, pbtp, zptp)

    col1.append(mg)
    col2.append(mr)
    col3.append(mi)
    col4.append(mz)
    col5.append(my)
    col6.append(mt)
    col7.append(mtp)
mg = np.array(col1)
mr = np.array(col2)
mi = np.array(col3)
mz = np.array(col4)
my = np.array(col5)
mt = np.array(col6)
mtp = np.array(col7)
# select only useful objects
good = (~np.isnan(mg)* ~np.isnan(mr)* ~np.isnan(mi)* ~np.isnan(mz)
        * ~np.isnan(my)* ~np.isnan(mt)* ~np.isnan(mtp))
good = np.array(good)

In [42]:
mg

array([-43.46142642, -43.42811488, -43.39254864, -43.35378539,
       -43.31022614, -43.26048351, -43.20377571, -43.13966557,
       -42.99956641, -42.6287441 , -42.02445073, -41.66027314,
       -41.14708305, -40.6098914 , -40.51962211, -40.10984049,
       -40.03337363, -39.39553985, -39.17058288, -38.81330978,
       -38.6054405 , -38.44856291, -38.28611731, -38.19172114,
       -37.98027733, -37.59440339, -36.91563952, -36.53442878,
       -36.13159024, -35.50615821, -35.04016818, -34.39970354,
       -34.22357047, -42.96758772, -41.61562478, -38.30378662,
       -37.4812987 , -36.88234632, -35.21675522, -34.76569469,
       -42.75658798, -41.41832794, -40.65280075, -40.28318691,
       -39.89888734, -39.34488903, -37.99695088, -37.04183687,
       -36.3062301 ])

In [47]:
pbs

OrderedDict([('tess',
              (<pysynphot.spectrum.ArraySpectralElement at 0x132687e6d8>,
               -21.8904558292275)),
             ('tessps1',
              (<pysynphot.spectrum.ArraySpectralElement at 0x1326880978>,
               -21.88238391456467)),
             ('ps1g',
              (<pysynphot.spectrum.ArraySpectralElement at 0x13264cffd0>,
               -20.836195000114955)),
             ('ps1r',
              (<pysynphot.spectrum.ArraySpectralElement at 0x132594a080>,
               -21.370261319386927)),
             ('ps1i',
              (<pysynphot.spectrum.ArraySpectralElement at 0x1325936dd8>,
               -21.793287528964058)),
             ('ps1z',
              (<pysynphot.spectrum.ArraySpectralElement at 0x132687c160>,
               -22.09903034378283)),
             ('ps1y',
              (<pysynphot.spectrum.ArraySpectralElement at 0x1324967cf8>,
               -22.325516493196154))])

In [48]:
-zpg

20.836195000114955